In [5]:
import requests
import gc
gc.collect()
import pandas as pd
import requests
import torch
print(torch.cuda.is_available())
from sklearn.model_selection import train_test_split
df = pd.read_csv('../data/測試數據/0927到2022.csv',index_col=0)
print(df.shape)
df[['45A']].head()

False
(2427, 22)


,45A
7,48 MT PVC SUSPENSION RESIN GRADE S-65\r\nAT TH...
9,TETRAHYDROFURAN AT USD 5550 TOTAL AMOUNT-USD 1...
10,A) 198.00 M.T. OF TAIRILAC AG15A1-H(ABS) ABS ...
11,80 MT PVC SUSPENSION RESIN GRADE S-65S AT THE ...
12,"PVC SUSPENSION RESIN S-70R\r\nQUANTITY: 72 MT,..."


# 載入模型

# 前處理

# 推論

# 展示結果